# Readme
+ Section 1 of this notebook trains the GAN on the optimization solutions, and outputs the initial solutions using the generator. Run section 1 after inputting the name of the experiment you would like to run at the beginning of section 1.2.
    + The generated raw solutions along with the plots of accuracy and loss are stored in ./CGAN/images
    + The input data for the selected experiment is stored in ./CGAN/input_data/
    + The final TF model is stored in ./CGAN/saved_model/

+ Move the data you like to analyze from ./CGAN/images/ to ./Data/

+ Section 2 of this notebook runs the evaluation function on the generated solutions, and outputs the stats and metrics associated with the generated solutions. Run section 2 after:
    + determining the name(s) of the experiment(s) and the configurations in lines 3-9 of section 2.3 for running the evaluation function,
    + determining the name(s) of the experiment(s) and the configurations in lines 15-17 of section 2.4 for calculating the hypervolumes, and
    + making sure the filenames listed in lines 2-15 of section 2.2 are those you have moved from ./CGAN/images/ to ./Data/.
    
+ You can find the complementary solutions in ./Data/ and the plots in ./Figures/. The rest of the stats are printed to the console.

# 1 Train the C-GAN and output the initial solutions

## 1.1 Initialize

In [ ]:
'''
Taken from https://github.com/eriklindernoren/Keras-GAN/tree/master/cgan

Modified by Author
Last modified on 1/22/2021
'''

from __future__ import print_function, division

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam


import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

from collections import deque

In [ ]:
import os
os.chdir('./CGAN')

## 1.2 Main (Input Data and Train CGAN)

In [ ]:
#  Set a name to distinguish the generated logs, visualizations, and models
run_name = 'WorstHalfAll'
# Determine the name of the experiment you want to run
experiment = 'WorstHalfAll'
# Determine if you want the long runs or short runs
long_run = True

### 1.2.1 Input Data

In [ ]:
# Constants for loading the dataset
Num_Sites = 4

LCC_Var = Num_Sites+11
CO2_Var = Num_Sites+14
WalkScore_Var = Num_Sites+15
GFA_Var = Num_Sites+16
FAR_Var = Num_Sites+17

PercentArea_0 = Num_Sites+19
PercentArea_1 = Num_Sites+27

Max_FAR = 20
Max_Site_GFA = 647497/Max_FAR # m2

error_tol = 1.15


# Import and filter the file `filename` based on several conditions
def DF_Filter(filename): # Similar to the one in Plots_Paper_One.py               
    file = np.loadtxt(filename, dtype='float', delimiter=',')
    inputDF = pd.DataFrame(file)

    
    
    print('+++++ processing %s +++++\n'%(filename))
    
    print('Count duplicates:')
    condition1 = inputDF.duplicated()==True
    print(inputDF[condition1][GFA_Var].count())
    
    print('Count under the min GFA:') # Count non-trivial neighborhoods
    condition2 = inputDF[GFA_Var] <= 1/error_tol#<=647497/10
    print(inputDF[condition2][GFA_Var].count())
    
    print('Count over the max GFA:')
    condition3 = inputDF[GFA_Var] >= Max_Site_GFA*Max_FAR*error_tol
    print(inputDF[condition3][GFA_Var].count())
    
    print('Count over the max Site GFA:')
    condition4 = inputDF[GFA_Var]/inputDF[FAR_Var] >= Max_Site_GFA*error_tol
    print(inputDF[condition4][GFA_Var].count())


    # Normalizing the LCC and CO2 objectives
    print('Normalizing the LCC and CO2 Obj Fxns')
    inputDF[LCC_Var] /= inputDF[GFA_Var] # Normalizing LCC ($/m2)
    inputDF[CO2_Var] /= inputDF[GFA_Var] # Normalizing CO2 (Tonnes/m2)
    print('Converting percent areas from decimal into percentage')
    for i in range(PercentArea_0, PercentArea_1+1): # Converting percent areas to integer %
        inputDF[i] = inputDF[i] * 100


    # Filter the data based on the 50-th percentiles of the objective function values
    if experiment == 'WorstHalfLCC':
        print('Permit only the worst 50% of individuals in terms of LCC:')
        LCC_percentile = np.nanpercentile(inputDF[LCC_Var], 50)
        print("Lowest LCC considered in training data:%.2f"%LCC_percentile)
        condition8 = (inputDF[LCC_Var] >= LCC_percentile)

    elif experiment == 'BestHalfLCC':
        print('Permit only the best 50% of individuals in terms of LCC:')
        LCC_percentile = np.nanpercentile(inputDF[LCC_Var], 50)
        print("Highest LCC considered in training data:%.2f"%LCC_percentile)
        condition8 = (inputDF[LCC_Var] <= LCC_percentile)
        
    elif experiment == 'WorstHalfCO2':
        print('Permit only the worst 50% of individuals in terms of CO2:')
        CO2_percentile = np.nanpercentile(inputDF[CO2_Var], 50)
        print("Lowest CO2 considered in training data: %.2f"%CO2_percentile)
        condition8 = (inputDF[CO2_Var] >= CO2_percentile)
        
    elif experiment == 'WorstHalfWalkScore':
        print('Permit only the worst 50% of individuals in terms of WalkScore:')
        WalkScore_percentile = np.nanpercentile(inputDF[WalkScore_Var], 50)
        print("Highest Walkscore considered in training data:%.2f"%WalkScore_percentile)
        condition8 = (inputDF[WalkScore_Var] <= WalkScore_percentile)

    elif experiment == 'WorstHalfAll':
        print('Permit only the worst 50% of individuals in terms of all objective functions:')
        LCC_percentile = np.nanpercentile(inputDF[LCC_Var], 50)
        CO2_percentile = np.nanpercentile(inputDF[CO2_Var], 50)
        WalkScore_percentile = np.nanpercentile(inputDF[WalkScore_Var], 50)
        condition8 = (inputDF[LCC_Var] >= LCC_percentile) & (inputDF[CO2_Var] >= CO2_percentile) & (inputDF[WalkScore_Var] <= WalkScore_percentile)
    
    elif experiment == 'BestHalfAll':
        print('Permit only the best 50% of individuals in terms of all objective functions:')
        LCC_percentile = np.nanpercentile(inputDF[LCC_Var], 50)
        CO2_percentile = np.nanpercentile(inputDF[CO2_Var], 50)
        WalkScore_percentile = np.nanpercentile(inputDF[WalkScore_Var], 50)
        condition8 = (inputDF[LCC_Var] <= LCC_percentile) & (inputDF[CO2_Var] <= CO2_percentile) & (inputDF[WalkScore_Var] >= WalkScore_percentile)


    elif experiment == 'BestHalfAny':
        print('Permit only the best 50% of individuals in terms of each objective functions:')
        LCC_percentile = np.nanpercentile(inputDF[LCC_Var], 50)
        CO2_percentile = np.nanpercentile(inputDF[CO2_Var], 50)
        WalkScore_percentile = np.nanpercentile(inputDF[WalkScore_Var], 50)
        condition8 = (inputDF[LCC_Var] <= LCC_percentile) | (inputDF[CO2_Var] <= CO2_percentile) | (inputDF[WalkScore_Var] >= WalkScore_percentile)



    # Filtering the inadmissible results
    Filtered = ~(condition1 | condition2 | condition3 | condition4)
    if experiment == 'FullData':
      inputDF = inputDF[Filtered]
    else:
      inputDF = inputDF[Filtered & condition8]


    if experiment == 'WorstHalfAll' or experiment == 'BestHalfAll' or experiment == 'BestHalfAny':
        print("Lowest LCC considered in training data:%.2f"%np.min(inputDF[LCC_Var]))
        print("Lowest CO2 considered in training data: %.2f"%np.min(inputDF[CO2_Var]))
        print("Highest Walkscore considered in training data:%.2f"%np.max(inputDF[WalkScore_Var]))    


    
    print('Count of valid answers: %d out of %d'%(len(inputDF), len(file)))
    inputDF.reset_index(inplace=True, drop=True)
    
    return inputDF


# Convert the raw input data into training features and labels vectors
def load_data():
    ## IMPORT DATA       
    print('loading data')
    filenames = ['../IILP_Toy_Optimization_TestRuns.txt']
    # DFNames = ['CCHP+Network']
    DF = DF_Filter(filenames[0])

    ## Set the range of input variables for the NN, then import the train variables
    inputRange = list(range(Num_Sites+8+1))
    inputRange.remove(Num_Sites) # plant location: the building type at plant location is already Num_Buildings + 1
    inputRange.remove(Num_Sites+3) # 0
    inputRange.remove(Num_Sites+4) # 0
    # Define X
    X = DF.iloc[:,inputRange]
    # Define y
    outputRange = [LCC_Var, CO2_Var, WalkScore_Var] # THESE ARE NORMALIZED IN DF_Filter
    y = DF.iloc[:, outputRange]
    


    
    print('Scaling data')
    # Scale X
    x = X.values #returns a numpy array
    std_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    X_Scaler = std_scaler.fit(x)
    X = X_Scaler.transform(x)
    # X = std_scaler.fit_transform(x)    
    
    # Scale y
    y_values = y.values #returns a numpy array
    std_scaler2 = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    y_Scaler = std_scaler2.fit(y_values)
    y = y_Scaler.transform(y_values)

    # Experiment: not scaling y: FAILED

    X_test = None
    y_test = None
    X_train = X
    y_train = y
    
    ## Reshape the data to fit the NN
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1).astype('float32')
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1).astype('float32')
    print('Scaled data')


    print('Saved training data in current directory as %s'%(run_name+'.txt'))
    np.savetxt('input_data/'+run_name+'.txt', DF.iloc[:, inputRange+outputRange])
    return X_train, y_train, X_test, y_test, X_Scaler, y_Scaler


X_train, y_train, X_test, y_test, X_Scaler, y_Scaler = load_data()

### 1.2.2 Create and train the CGAN on the data

In [ ]:
class CGAN():
    def __init__(self, X_train, y_train, X_test, y_test, X_Scaler, y_Scaler, latent_dim=3):
        # Input shape
        # self.img_rows = 1
        self.img_cols = len(X_train[0]) # Was 11 before Nov 5, 2020
        self.label_size = len(y_train[0])
        self.channels = 1
        self.img_shape = (self.img_cols, self.channels) #(self.img_rows, self.img_cols, self.channels)
        # self.num_classes = 10
        self.latent_dim = latent_dim # Was 22


        optimizer = Adam(learning_rate = 0.0002, beta_1 = 0.5, beta_2 = 0.999) # Adam(0.0002, 0.5)


        self.X_train, self.y_train, self.X_Scaler, self.y_Scaler = \
            X_train, y_train, X_Scaler, y_Scaler



        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(self.label_size,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)



    def build_discriminator(self):
        # Best architecture: 1. 128-64-32 2. 256-256-128, 3. 512-256-128; 4. 512-512-512

        model = Sequential()

        model.add(Dense(128, input_dim=np.prod(self.img_shape)+self.label_size))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(64))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(32))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(self.label_size,), dtype='float32')

        # label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        # model_input = multiply([flat_img, label_embedding])
        model_input = Concatenate(axis=1)([flat_img, label])

        validity = model(model_input)

        return Model([img, label], validity)



    def build_generator(self):
      # Best architecture so far:
      # 1. 64-128-64; 2. 128-256-128; 3. 256 - 512 - 256

        model = Sequential()

        model.add(Dense(64, input_dim=self.latent_dim+self.label_size))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(64))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        # model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(self.label_size,), dtype='float32')
        # label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        # model_input = multiply([noise, label_embedding])
        model_input = Concatenate(axis=1)([noise, label])
        img = model(model_input)

        return Model([noise, label], img)

    
  


    def train(self, epochs=1000, batch_size=128, sample_interval=200, runningAvgDur=10):
        # Load the dataset
        X_train, y_train, X_Scaler, y_Scaler =\
            self.X_train, self.y_train, self.X_Scaler, self.y_Scaler
        # X_train, y_train, X_test, y_test, X_Scaler, y_Scaler = self.load_data()


        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        d_losses_real = []
        d_accs_real = []
        d_losses_fake = []
        d_accs_fake = []

        g_losses = []

        startOfDur = -int(runningAvgDur)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            # d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels

            sampled_indx = np.random.choice(y_train.shape[0], size=batch_size)  #np.random.randint(0, 10, batch_size).reshape(-1, 1)
            sampled_labels = y_train[sampled_indx]

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)


            # Register the progress
            d_losses_real.append(d_loss_real[0])
            d_accs_real.append(100*d_loss_real[1])

            d_losses_fake.append(d_loss_fake[0])
            d_accs_fake.append(100*d_loss_fake[1])

            g_losses.append(g_loss)

            



            # If at save interval => save generated image samples
            if (epoch > 0):
              if (epoch % sample_interval == 0) or (epoch == epochs - 1): #  and epoch >= 3*epochs/4
                self.sample_images(epoch)
                print("%d [D loss_real: %.3f, acc_real: %.2f%%] [D loss_fake: %.3f, acc_fake: %.2f%%] [G loss: %.3f]"
                  % (epoch, np.average(d_losses_real[startOfDur:]),
                     np.average(d_accs_real[startOfDur:]),
                     np.average(d_losses_fake[startOfDur:]),
                     np.average(d_accs_fake[startOfDur:]),
                     np.average(g_losses[startOfDur:])))
        

        self.plot_results(d_losses_real, d_losses_fake, g_losses, d_accs_real,
                          d_accs_fake, runningAvgDur, run_name, epochs)


        # Save the trained models
        self.generator.save('saved_model/'+run_name+'_generator_model.tf')
        self.discriminator.save('saved_model/'+run_name+'_discriminator_model.tf')
        self.combined.save('saved_model/'+run_name+'_combined_model.tf')



    def sample_images(self, epoch):
        if experiment != 'FullData' and experiment != 'BestHalfAll' and experiment != 'WorstHalfAll':
          r, c = 125, 1
        else:
          r, c = 4*125, 1

        noise = np.random.normal(0, 1, (r * c, self.latent_dim))


        sampled_labels = []

        # Generate the random samples [i: LCC, j: GHG, k: WalkScore]
        if experiment == 'WorstHalfLCC':
          for i in np.arange(-1.0, -1.5, -0.1):
            for j in np.arange(-1.0, 1.5, 0.5):
              for k in np.arange(-1.0, 1.5, 0.5):
                sampled_labels.append([i,j,k])
        

        elif experiment == 'WorstHalfCO2':
          for i in np.arange(-1.0, 1.5, 0.5):
            for j in np.arange(-1.0, -1.25, -0.05):
              for k in np.arange(-1.0, 1.5, 0.5):
                sampled_labels.append([i,j,k])

        elif experiment == 'WorstHalfWalkScore':
          for i in np.arange(-1.0, 1.5, 0.5):
            for j in np.arange(-1.0, 1.5, 0.5):
              for k in np.arange(1.0, 1.5, 0.1):
                sampled_labels.append([i,j,k])
        

        elif experiment == 'FullData' or experiment == 'BestHalfAll'\
              or experiment == 'BestHalfAny' or experiment == 'WorstHalfAll':
          for i in np.arange(-1.0, -1.5, -0.1):
            for j in np.arange(-1.0, 1.5, 0.5):
              for k in np.arange(-1.0, 1.5, 0.5):
                sampled_labels.append([i,j,k])

          for i in np.arange(-1.0, 1.5, 0.5):
            for j in np.arange(-1.0, -1.25, -0.05):
              for k in np.arange(-1.0, 1.5, 0.5):
                sampled_labels.append([i,j,k])
          
          for i in np.arange(-1.0, 1.5, 0.5):
            for j in np.arange(-1.0, 1.5, 0.5):
              for k in np.arange(1.0, 1.5, 0.1):
                sampled_labels.append([i,j,k])

          for i in np.arange(-1.0, -1.5, -0.1):
            for j in np.arange(-1.0, -1.25, -0.05):
              for k in np.arange(1.0, 1.5, 0.1):
                sampled_labels.append([i,j,k])



        gen_imgs = self.generator.predict([noise, np.array(sampled_labels)])
        scaled_gen_imgs = self.X_Scaler.inverse_transform(gen_imgs)

        np.savetxt("images/"+run_name+"_%d.txt" % epoch, scaled_gen_imgs)



    def plot_results(self, d_losses_real, d_losses_fake, g_losses, d_accs_real,
                     d_accs_fake, runningAvgDur, run_name, epochs):
      # Plot the result of the model
      plt.style.use('ggplot')
      
      plt.figure()
      plt.plot(range(epochs), d_losses_real, color='red', alpha=0.1, label='Disc Loss Real')
      d_losses_real_avg = self.moving_average(d_losses_real, runningAvgDur)
      plt.plot(range(epochs), d_losses_real_avg, color='red', label='Disc Loss Real_Avg')

      plt.plot(range(epochs), d_losses_fake, color='green', alpha=0.1, label='Disc Loss Fake')
      d_losses_fake_avg = self.moving_average(d_losses_fake, runningAvgDur)
      plt.plot(range(epochs), d_losses_fake_avg, color='green', label='Disc Loss Fake_Avg')

      plt.plot(range(epochs), g_losses, color='blue', alpha=0.1, label='Gen Loss')
      g_losses_avg = self.moving_average(g_losses, runningAvgDur)
      plt.plot(range(epochs), g_losses_avg, color='blue', label='Gen Loss_Avg')

      plt.title('Discriminator & Generator Loss vs Iteration Number')
      plt.xlabel('Iteration #')
      plt.ylabel('Loss')
      plt.legend()
      plt.savefig('images/'+run_name+'_Disc Loss vs Gen Loss.png', bbox_inches='tight', dpi=400)
      plt.show()

      plt.figure()
      plt.plot(range(epochs), d_accs_real, color='red', alpha=0.1, label='Disc Acc Real')
      d_accs_real_avg = self.moving_average(d_accs_real, runningAvgDur)
      plt.plot(range(epochs), d_accs_real_avg, color='red', label='Disc Acc Real_Avg')

      plt.plot(range(epochs), d_accs_fake, color='green', alpha=0.1, label='Disc Acc Fake')
      d_accs_fake_avg = self.moving_average(d_accs_fake, runningAvgDur)
      plt.plot(range(epochs), d_accs_fake_avg, color='green', label='Disc Acc Fake_Avg')

      plt.title('Discriminator Accuracy vs Iteration Number')
      plt.xlabel('Iteration #')
      plt.ylabel('Accuracy \%')
      plt.legend()
      plt.savefig('images/'+run_name+'_Disc Acc.png', bbox_inches='tight', dpi=400)
      plt.show()


    def moving_average(self, a, n=3):
        # Calculate the moving average of array a with window_size n
        ret = np.cumsum(a, dtype=float)
        ret[n:] = ret[n:] - ret[:-n]
        ret[:n-1] /= np.arange(1,n)
        ret[n - 1:] /= n
        return ret


In [ ]:
# Train the CGAN and generate outputs
latent_dim = 3
cgan = CGAN(X_train, y_train, X_test, y_test, X_Scaler, y_Scaler, latent_dim=latent_dim)
# Long runs
if long_run:
    if experiment in ['WorstHalfLCC', 'WorstHalfCO2']: # ~155 epochs
      cgan.train(epochs=20000, batch_size=256, sample_interval=1000, runningAvgDur=10)
    elif experiment in ['WorstHalfWalkScore']: # ~155 epoch
      cgan.train(epochs=24000, batch_size=256, sample_interval=1000, runningAvgDur=10) 
    elif experiment == 'FullData':# ~156 epochs
      cgan.train(epochs=40000, batch_size=256, sample_interval=1000, runningAvgDur=10) # 256 iterations ~= 1 epoch for the full data # Tried at first batch of 64 over 10,000 epochs, it was good!
    elif experiment == 'WorstHalfAll':# ~155 epochs
      cgan.train(epochs=3200, batch_size=256, sample_interval=100, runningAvgDur=10) 
    elif experiment == 'BestHalfAll':# ~155 epochs
      cgan.train(epochs=5000, batch_size=256, sample_interval=150, runningAvgDur=10) 
# Short runs:
else:
    if experiment in ['WorstHalfAll', 'BestHalfAll', 'FullData']:# ~16, 25 epochs
      cgan.train(epochs=2000, batch_size=64, sample_interval=100, runningAvgDur=10) 
    elif experiment in ['WorstHalfWalkScore', 'WorstHalfLCC', 'WorstHalfCO2']: # ~1 epoch
      cgan.train(epochs=800, batch_size=64, sample_interval=100, runningAvgDur=10) 

## Auxilliary: Generate more images using a saved TF model


In [ ]:
'''
from tensorflow.keras.models import load_model
os.chdir('./CGAN/')

def sample_images(generator, epoch, y_Scaler, X_Scaler):
  r, c = 125, 1
  noise = np.random.normal(0, 1, (r * c, latent_dim))

  sampled_labels = []

  # Experiment: getting the individuals with the best LCC
  for i in np.arange(-1.0, -1.25, -0.05):
    for j in np.arange(-1.0, 1.5, 0.5):
      for k in np.arange(-1.0, 1.5, 0.5):
        sampled_labels.append([i,j,k])
  

  gen_imgs = generator.predict([noise, np.array(sampled_labels)])
  scaled_gen_imgs = X_Scaler.inverse_transform(gen_imgs)

  np.savetxt("images/"+run_name+"_%d.txt" % epoch, scaled_gen_imgs)


generator = load_model(filepath='./saved_model/'+run_name+'_generator_model.tf')
sample_images(generator, epoch=1, y_Scaler=y_Scaler, X_Scaler=X_Scaler)
'''

# 2 Evaluate the generated solutions

## 2.1 Initialize

In [ ]:
os.chdir('..')

In [ ]:
# Constants for plotting the filtered plots
LCC_Cutoff = 100 # k$/m2
CO2_Cutoff = 10 # T-CO2/m2



# Constants for evaluating the solutions
Num_Sites = 4
Num_Buildings = 4
len_of_indiv = 11 # After adding the plant location


LCC_Var = Num_Sites+11
CO2_Var = Num_Sites+14
WalkScore_Var = Num_Sites+15

Num_Chillers = 3#2                           # Must be updated if new chillers are added ## NOTE: changed from 16 to 2
Num_Engines = 6
Building_Min = 0
Supply_Min = 1


Max_Supply_Temp_Heating = 95.0              # deg C
Min_Supply_Temp_Heating = 50.0              # deg C
Min_Summer_Heating_Reset_del_T = 0          # deg C
Min_Winter_Cooling_Reset_del_T = 0          # deg C
Max_Summer_Heating_Reset_del_T = 10         # deg C
Max_Winter_Cooling_Reset_del_T = 3          # deg C
Max_Supply_Temp_Cooling = 8.0               # deg C
Min_Supply_Temp_Cooling = 1.0               # deg C




# Load the libraries
import sys
import os
Ch3_Dir = './Simulation/'
Curr_Dir = os.getcwd()
sys.path.append(Ch3_Dir)

os.chdir(Ch3_Dir)
from RQ3 import SupplyandDemandOptimization as analyze
os.chdir(Curr_Dir)

import numpy as np
from Filter_Data import DF_Filter
from matplotlib import pyplot as plt

from tqdm import tqdm



## Check the validity of the inputs
Low_Seq = []
High_Seq = []
for i in range(Num_Sites):
    Low_Seq += [Building_Min]
    High_Seq += [Num_Buildings]
   
# Create the Low and High Sequence values that control the location of the Central Plant
Low_Seq += [0]
High_Seq += [Num_Sites-1]

# Create the Low and High Sequence values that control mutation in optimization for CHP engines
Low_Seq += [Supply_Min]
High_Seq += [Num_Engines]

# Create the Low and High Sequence values that control mutation in optimization for the chillers
Low_Seq += [Supply_Min]
High_Seq += [Num_Chillers]

# Create the Low and High Sequence values that control mutation for loop temperature and thermal reset
Low_Seq += [Min_Supply_Temp_Heating]
Low_Seq += [Min_Summer_Heating_Reset_del_T]
Low_Seq += [Min_Supply_Temp_Cooling]
Low_Seq += [Min_Winter_Cooling_Reset_del_T]
High_Seq += [Max_Supply_Temp_Heating]
High_Seq += [Max_Summer_Heating_Reset_del_T]
High_Seq += [Max_Supply_Temp_Cooling]
High_Seq += [Max_Winter_Cooling_Reset_del_T]

## 2.2 Helper functions

In [ ]:
## Helper function for loading the filenames of the results
def load_filenames(Experiment):
    if Experiment == 'FullData':
        fileNames = ['FullData_1000', 'FullData_5000', 'FullData_15000', 'FullData_39999'] 
    elif Experiment == 'WorstHalfLCC': 
        fileNames = ['WorstHalfLCC_1000', 'WorstHalfLCC_5000', 'WorstHalfLCC_10000', 'WorstHalfLCC_15000', 'WorstHalfLCC_19999']
    elif Experiment == 'WorstHalfCO2': 
        fileNames = ['WorstHalfCO2_1000', 'WorstHalfCO2_5000', 'WorstHalfCO2_10000', 'WorstHalfCO2_15000', 'WorstHalfCO2_19999']
    elif Experiment == 'WorstHalfWalkScore':
        fileNames = ['WorstHalfWalkscore_1000', 'WorstHalfWalkscore_5000', 'WorstHalfWalkscore_10000', 'WorstHalfWalkscore_15000', 'WorstHalfWalkscore_23999']
    elif Experiment == 'WorstHalfAll':
        fileNames = ['WorstHalfAll2_100', 'WorstHalfAll2_300', 'WorstHalfAll2_500', 'WorstHalfAll2_1500', 'WorstHalfAll2_3199']
    elif Experiment == 'BestHalfAll':
        fileNames = ['BestHalfAll_150', 'BestHalfAll_600', 'BestHalfAll_2100', 'BestHalfAll_4050', 'BestHalfAll_4999']
    return fileNames
        
        
## Helper functions for combining the results of the long and short runs
def results_total_finder(experiment):
    path = './Data/'
    files = []
    for i in os.listdir(path):
        if os.path.isfile(os.path.join(path,i)) and 'resultsTotal'+experiment in i:
            files.append('./Data/'+i)
    return files



def load_combined_results(experiment):
    allResults = None
    for i, filename in enumerate(results_total_finder(experiment)):
        if i == 0:
            allResults = np.loadtxt(filename)
        else:
            allResults = np.append(allResults, np.loadtxt(filename), axis=0)
        print('processed %s'%filename)
        
    print('Loaded %d points generated by GAN from combined results of %s'%(len(allResults), experiment))
    return allResults

## 2.3 Main

In [ ]:
# DECLARE THE EXPERIMENT TYPE
plot_only = False # If you want no evaluation and simply plotting the results, set this to True; no plots are made when this is False
stats_and_plot = False # If you want statistical info to be printed out while plot_only is True, set this to True
aggregate_results = False # If you want the results of the long and short runs to get combined, set this to be True
verbose = True # If you want full outputs to be printed out, set this to True

for Experiment in ['WorstHalfCO2', 'WorstHalfLCC', 'WorstHalfWalkScore',
                    'WorstHalfAll', 'BestHalfAll', 'FullData']: ## MODIFY THIS TO RUN DIFFERENT EXPERIMENTS
    plt.close('all')
    # Load the proper set of filenames based on the experiment
    fileNames = load_filenames(Experiment)

    
    # Read in the data generated by GAN or load the already processed dataset
    if not plot_only:
        if not os.path.exists('./Data/rebuilt_'+fileNames[0]+'.txt'):
            # If more than one filename provided, combine them and save them as one
            loadedData = np.round(np.loadtxt('./Data/'+fileNames[0]+'.txt'))
            for name in fileNames[1:]:
                loadedData = np.append(loadedData, np.round(np.loadtxt('./Data/'+name+'.txt')), axis=0)
                
            np.savetxt('./Data/rebuilt_'+fileNames[0]+'.txt', loadedData)
            print('Saved rebuilt data generated by GAN into %s'%('./Data/rebuilt_'+fileNames[0]+'.txt'))
        else:
            loadedData = np.loadtxt('./Data/rebuilt_'+fileNames[0]+'.txt')
            print('Loaded rebuilt data generated by GAN from %s'%('./Data/rebuilt_'+fileNames[0]+'.txt'))

    
    
    
    
    # Evaluate the individuals or load the already-evaluated individuals
    if not aggregate_results and not os.path.exists('./Data/resultsTotal'+fileNames[0]+'.txt'):
        # Evaluate the individuals
        resultsTotal = []
        correctedIndivs = 0 # Number of individuals corrected (meaning they were outside the allowed boundary)
        ignoredIndivs = 0 # Number of ignored individuals
    
        for indiv in tqdm(loadedData):
            indiv = list(indiv)
            # Fit the generated individual inside the High_Seq - Low_Seq range
            oldIndiv = indiv
            
            # Find the plant location and insert it into the individual's DNA
            plantLoc = None
            for i in range(4):
                if indiv[i] >= Num_Buildings: # Found potential plant location
                    if plantLoc == None: # First time finding such location
                        plantLoc = i
                        indiv[i] = 0 # Setting the building variable in plant location to zero
                    else: # Already found the plant location
                        indiv[i] = 0 # Invalid plant location, removing the building in its place
                        
            if plantLoc == None: # Not found any plant locations!
                ignoredIndivs += 1
                continue
            else: # Insert plant location into the individual's DNA
                indiv.insert(Num_Sites, plantLoc)
            
            # Check the validity of the DNA and round to the nearest valid number
            indiv = list(np.minimum(np.maximum(indiv, Low_Seq), High_Seq))
            
            # Check if there are any buildings in the development
            if np.sum(indiv[:4]) == 0:
                ignoredIndivs += 1
                continue
            
            # Register if the individual has been corrected
            if indiv != oldIndiv:
                correctedIndivs += 1
    
            results = analyze(indiv, solver='gurobi')
    
            # Skip trivial cases
            if float('inf') in results[0] or -float('inf') in results[0]: 
                ignoredIndivs += 1
                continue
    
            [LCC_per_GFA, CO2_per_GFA, WalkScore,] = results[0]
            if LCC_per_GFA != float('inf'):
                indiv.extend(results[0])
                resultsTotal.append(indiv)
    
        print('\n\n++++++++++++++++++++++++++\nNumber of corrected individuals: %d out of %d'%(correctedIndivs, len(loadedData)))
        print('Number of ignored individuals: %d out of %d'%(ignoredIndivs, len(loadedData)))
    
        np.savetxt('./Data/resultsTotal'+fileNames[0]+'.txt', resultsTotal)
        print('Saved points generated by GAN and their associated results into %s'%('./Data/resultsTotal'+fileNames[0]+'.txt'))
    
    else:
        if not aggregate_results:
            resultsTotal = np.loadtxt('./Data/resultsTotal'+fileNames[0]+'.txt')
            print('Loaded points generated by GAN from %s'%('./Data/resultsTotal'+fileNames[0]+'.txt'))
        else:
            resultsTotal = load_combined_results(Experiment)

    resultsTotal = np.array(resultsTotal)
    
    
    
    
    
    
    if verbose: print('loading data generated by the optimization algorithm')
    filename = './IILP_Toy_Optimization_TestRuns.txt'
    DFName = 'Initial'
    if plot_only or stats_and_plot:
        DF = DF_Filter(filename, experiment=Experiment, verbose=0)
    else:
        DF = DF_Filter(filename, experiment=Experiment)#, verbose=0)
    
    
    plt.style.use('ggplot')
    colors_rb = {DFName:'r'}
    
    #############################################
    
    if plot_only or stats_and_plot:
        if verbose: print('\nPlotting!\n')
    
    
        # LCC vs CO2
        if verbose: print('Plotting LCC vs CO2')
        plt.figure(figsize=(10,5))
        plt.scatter(x=DF[LCC_Var]/10**3,y=DF[CO2_Var], label=DFName, s=100, alpha=1, c=colors_rb[DFName], marker='x')
        plt.scatter(x=resultsTotal[:,len_of_indiv+0]/10**3, y=resultsTotal[:,len_of_indiv+1], label='C-GAN', s=150, alpha=1, c='blue', marker='+')
        
        plt.xlabel(r'LCC (k\$/$m^2$)')
        plt.ylabel(r'GHG (T-$CO_{2e}$/$m^2$)')
        plt.legend()
        plt.savefig('./Figures/All_CO2_vs_LCC'+fileNames[0]+'.png', dpi=400, bbox_inches='tight')
        
        
        
        # LCC vs Walkscore
        if verbose: print('Plotting LCC vs Walkscore')
        plt.figure(figsize=(10,5))
        # DF = DF[DF[LCC_Var]/10**3 <= 500]
        # DF = DF[DF[Walk] <= 100] # Observation: with a less than $500k/m2 of LCC, no designs have CO2/m2 higher than 5 T-CO2/m2
        plt.scatter(x=DF[LCC_Var]/10**3,y=DF[WalkScore_Var], label=DFName, s=100, alpha=1, c=colors_rb[DFName], marker='x')
        plt.scatter(x=resultsTotal[:,len_of_indiv+0]/10**3,y=resultsTotal[:,len_of_indiv+2], label='C-GAN', s=150, alpha=1, c='blue', marker='+')
        
        plt.xlabel(r'LCC (k\$/$m^2$)')
        plt.ylabel(r'Walkscore')
        plt.legend()
        plt.savefig('./Figures/All_Walkscore_vs_LCC'+fileNames[0]+'.png', dpi=400, bbox_inches='tight')
    
    
    
    
    
   
    if not plot_only or stats_and_plot:
        ## FIND DUPLICATE ELEMENTS B/W The original and the synthetic individuals
        if verbose: print('\n\n+++++\nFind duplicated elements b/w the original and the synthetic individuals')
        original_data = np.array(DF.iloc[:,:13])
        original_data = np.concatenate((original_data[:,:7], original_data[:,9:]),axis=1)
        list_original_data = [list(item) for item in original_data]
        
        
        synth_data = resultsTotal[:,:11]
        list_synth_data = [list(item) for item in synth_data]
        
        duplicates = []
        for i in range(len(synth_data)):
            if list_synth_data[i] in list_original_data:
                duplicates.append(i)
                
        if verbose: print('Number of elements shared between the original and the synthetic data: %d'%len(duplicates))
        # if duplicates:
        #     print('List of elements shared between the original and the synthetic data (indices defined): %d'%len(duplicates))
    
    
    
    
    
    
        # Check the improvement in targeted objective function compared to the training data
        if Experiment == 'WorstHalfLCC' or Experiment == 'BestHalfLCC':
            lowest_generated = np.min(resultsTotal[:,len_of_indiv+0])
            print('lowest LCC in generated individuals: %.2f'%lowest_generated)
            print('--> Percent improvement: %.1f%%'%((lowest_generated-np.min(DF[LCC_Var]))/np.min(DF[LCC_Var])*100))
        elif Experiment == 'WorstHalfCO2':
            lowest_generated = np.min(resultsTotal[:,len_of_indiv+1])
            print('Lowest CO2 in generated data: %.2f'%lowest_generated)
            print('--> Percent improvement: %.1f%%'%((np.min(DF[CO2_Var])-lowest_generated)/np.min(DF[CO2_Var])*100))
        elif Experiment == 'WorstHalfWalkScore':
            highest_generated = np.max(resultsTotal[:,len_of_indiv+2])
            print('Highest Walkscore in generated data: %.2f'%highest_generated)
            print('--> Percent improvement: %.1f%%'%((highest_generated - np.max(DF[WalkScore_Var]))/np.max(DF[WalkScore_Var])*100))
        elif Experiment == 'WorstHalfAll' or Experiment == 'FullData' or Experiment == 'BestHalfAll' or Experiment == 'BestHalfAny':
            lowest_generated_LCC = np.min(resultsTotal[:,len_of_indiv+0])
            lowest_generated_CO2 = np.min(resultsTotal[:,len_of_indiv+1])
            highest_generated_WalkScore = np.max(resultsTotal[:,len_of_indiv+2])
            print('lowest LCC in generated individuals: %.2f'%lowest_generated_LCC)
            print('Lowest CO2 in generated data: %.2f'%lowest_generated_CO2)
            print('Highest Walkscore in generated data: %.2f'%highest_generated_WalkScore)
            print('--> Percent improvement in LCC: %.1f%%'%((lowest_generated_LCC-np.min(DF[LCC_Var]))/np.min(DF[LCC_Var])*100))
            print('--> Percent improvement in CO2: %.1f%%'%((np.min(DF[CO2_Var])-lowest_generated_CO2)/np.min(DF[CO2_Var])*100))
            print('--> Percent improvement in WalkScore: %.1f%%'%((highest_generated_WalkScore - np.max(DF[WalkScore_Var]))/np.max(DF[WalkScore_Var])*100))
        
    
    
    
    
    if plot_only or stats_and_plot:
        # LCC vs CO2 (Filtered below the cutoff thresholds)
        ## Filter dataframes based on max values of CO2 and LCC
        if verbose: print('++++\nFiltering DF and resultsTotal based on cutoffs (LCC_Cutoff: %d, CO2_Cutoff: %d)'%(LCC_Cutoff, CO2_Cutoff))
        DF = DF[DF[LCC_Var]/10**3 <= LCC_Cutoff]
        DF = DF[DF[CO2_Var] <= CO2_Cutoff] # Observation: with a less than $500k/m2 of LCC, no designs have CO2/m2 higher than 5 T-CO2/m2
        resultsTotal = resultsTotal[resultsTotal[:,len_of_indiv+0]/10**3 <= LCC_Cutoff]
        resultsTotal = resultsTotal[resultsTotal[:,len_of_indiv+1] <= CO2_Cutoff]
        
        
        
        if verbose: print('Plotting LCC vs CO2')
        plt.figure(figsize=(10,5))
        plt.scatter(x=DF[LCC_Var]/10**3,y=DF[CO2_Var], label=DFName, s=100, alpha=1, c=colors_rb[DFName], marker='x')
        plt.scatter(x=resultsTotal[:,len_of_indiv+0]/10**3, y=resultsTotal[:,len_of_indiv+1], label='C-GAN', s=150, alpha=1, c='blue', marker='+')
        
        plt.xlabel(r'LCC (k\$/$m^2$)')
        plt.ylabel(r'GHG (T-$CO_{2e}$/$m^2$)')
        plt.legend()
        plt.savefig('./Figures/CO2_vs_LCC'+fileNames[0]+'.png', dpi=400, bbox_inches='tight')
        
        
        
        
        # LCC vs Walkscore (Filtered below the cutoff thresholds)
        if verbose: print('Plotting LCC vs Walkscore')
        plt.figure(figsize=(10,5))
        plt.scatter(x=DF[LCC_Var]/10**3,y=DF[WalkScore_Var], label=DFName, s=100, alpha=1, c=colors_rb[DFName], marker='x')
        plt.scatter(x=resultsTotal[:,len_of_indiv+0]/10**3,y=resultsTotal[:,len_of_indiv+2], label='C-GAN', s=150, alpha=1, c='blue', marker='+')
        
        plt.xlabel(r'LCC (k\$/$m^2$)')
        plt.ylabel(r'Walkscore')
        plt.legend()
        plt.savefig('./Figures/Walkscore_vs_LCC'+fileNames[0]+'.png', dpi=400, bbox_inches='tight')

## 2.4 Calculate the changes in hypervolume

In [ ]:
# Constants
ref_point = [1.0, 1.0, 1.0] # Reference point for calculating the hypervolume


## Labels for the generated dataset
LCC_Var_Gen = len_of_indiv+0
CO2_Var_Gen = len_of_indiv+1
WalkScore_Var_Gen = len_of_indiv+2

np.random.seed(42)



# DECLARE THE EXPERIMENT TYPE
aggregate_results = True
for Experiment in ['WorstHalfCO2', 'WorstHalfLCC', 'WorstHalfWalkScore',
                    'WorstHalfAll', 'BestHalfAll', 'FullData']: ## MODIFY THIS TO RUN DIFFERENT EXPERIMENTS
    print('\n+++++++++++++++++')
    print('Results for the experiment %s:\n'%Experiment)
    
    # Load the proper set of filenames based on the experiment
    fileNames = load_filenames(Experiment)
    
    
    
    # Load the datasets
    if not aggregate_results:
        resultsTotal = pd.DataFrame(np.loadtxt('./Data/resultsTotal'+fileNames[0]+'.txt'))
        print('Loaded points generated by GAN from %s'%('./Data/resultsTotal'+fileNames[0]+'.txt'))
    else:
        resultsTotal = pd.DataFrame(load_combined_results(Experiment))

    filename = './IILP_Toy_Optimization_TestRuns.txt'
    DF = DF_Filter(filename, experiment=Experiment, verbose=0)
    
    
    
    # Calculate the hypervolume
    ## modify the input parameters to make them suitable for calculating the HV w.r.t. the reference point (1,1,1)
    maxLCC = max(np.max(DF[LCC_Var]), np.max(resultsTotal[LCC_Var_Gen]))
    maxCO2 = max(np.max(DF[CO2_Var]), np.max(resultsTotal[CO2_Var_Gen]))
    maxWalkScore = max(np.max(DF[WalkScore_Var]), np.max(resultsTotal[WalkScore_Var_Gen]))
    
    minLCC = min(np.min(DF[LCC_Var]), np.min(resultsTotal[LCC_Var_Gen]))
    minCO2 = min(np.min(DF[CO2_Var]), np.min(resultsTotal[CO2_Var_Gen]))
    minWalkScore = min(np.min(DF[WalkScore_Var]), np.min(resultsTotal[WalkScore_Var_Gen]))
    
    
    ## Reverse the walkscore column
    def reverse_data(DF, column, maxValue): # Subtract each value from the given max value in a column of a dataframe/np.array
        DF[column] = maxValue - DF[column]
        
    reverse_data(DF, WalkScore_Var, maxWalkScore)
    reverse_data(resultsTotal, WalkScore_Var_Gen, maxWalkScore)
    
    
    ## Normalize the data to 0 to 1
    def normalize_data(DF, column, minValue, maxValue):
        DF[column] = (DF[column] - minValue)/(maxValue - minValue)
    
    normalize_data(DF, LCC_Var, minLCC, maxLCC)
    normalize_data(DF, CO2_Var, minCO2, maxCO2)
    normalize_data(DF, WalkScore_Var, minWalkScore, maxWalkScore)
    
    normalize_data(resultsTotal, LCC_Var_Gen, minLCC, maxLCC)
    normalize_data(resultsTotal, CO2_Var_Gen, minCO2, maxCO2)
    normalize_data(resultsTotal, WalkScore_Var_Gen, minWalkScore, maxWalkScore)
    
    
    
    ## Calculate the hv
    from pymoo.factory import get_performance_indicator
    hv = get_performance_indicator("hv", ref_point=np.array(ref_point))
    
    
    ## GIVES MEMORY ERROR IF USED DIRECTLY ##
    array1 = np.array(DF[[LCC_Var, CO2_Var, WalkScore_Var]])
    
    
    array2 = np.array(resultsTotal[[LCC_Var_Gen, CO2_Var_Gen, WalkScore_Var_Gen]])
    generatedArea = hv.calc(array2)
    
    
    originalAreas = []
    # prevArr = None
    Num_Samplings = int(len(array1)/len(array2))+1
    for i in range(Num_Samplings):
        choices = np.random.randint(low=0, high=len(array1), size=len(array2))
        array1_2 = np.array(DF[[LCC_Var, CO2_Var, WalkScore_Var]])[choices, :]
        originalAreas.append(hv.calc(array1_2))
    
    meanOriginalArea = np.mean(originalAreas)
    maxOriginalArea = np.max(originalAreas)
    
      
    print('Maximum generated hv:%.2e'%generatedArea)
    print('Maximum original hv:%.2e'%maxOriginalArea)
    print('Mean original hv:%.2e'%meanOriginalArea)
    if generatedArea > meanOriginalArea:
        print('Generated solutions have on average a hv %.2f%% larger than the original solutions'%((generatedArea - meanOriginalArea)/meanOriginalArea*100))
    else:
        print('Original solutions have on average a hv %.2f%% larger than the generated solutions'%((meanOriginalArea - generatedArea)/meanOriginalArea*100))
        
        
    if generatedArea > maxOriginalArea:
        print('Generated solutions have a hv %.2f%% larger than the best of original solutions'%((generatedArea - maxOriginalArea)/maxOriginalArea*100))
    else:
        print('Original solutions have at best a hv %.2f%% larger than the generated solutions'%((maxOriginalArea - generatedArea)/maxOriginalArea*100))